In [2]:
import threading


In [3]:
def do_something():
    print("Look, I'm running in a thread!")

thread = threading.Thread(target=do_something)
thread.start()


Look, I'm running in a thread!


In [4]:
thread.join()


In [5]:
import threading

mutex = threading.Lock()
counter = 0

def increment():
    global counter
    for _ in range(1000000):
        mutex.acquire()
        counter += 1
        mutex.release()

t1 = threading.Thread(target=increment)
t2 = threading.Thread(target=increment)
t1.start()
t2.start()

t1.join()
t2.join()
print("Final counter:", counter)


Final counter: 2000000


In [6]:
import threading

rlock = threading.RLock()
counter = 0

def increment():
    global counter
    for _ in range(1000000):
        rlock.acquire()
        counter += 1
        rlock.release()

t1 = threading.Thread(target=increment)
t2 = threading.Thread(target=increment)
t1.start()
t2.start()
t1.join()
t2.join()
print("Final counter:", counter)


Final counter: 2000000


In [7]:
import threading
import time

event = threading.Event()
counter = 0

def worker():
    global counter
    while not event.is_set():
        print("Working...")
        time.sleep(1)
        counter += 1
    print("Event is set, stopping.")

t = threading.Thread(target=worker)
t.start()
time.sleep(3)
event.set()
t.join()
print("Final counter:", counter)	


Working...
Working...
Working...
Event is set, stopping.
Final counter: 3


In [8]:
import threading
import time

# 全局变量
counter = 0
mutex = threading.Lock()
event = threading.Event()

def worker():
    global counter
    while not event.is_set():
        mutex.acquire()
        counter += 1
        print(f"Counter: {counter}")
        mutex.release()
        time.sleep(1)
    print("Worker stopped.")

# 创建并启动线程
t = threading.Thread(target=worker)
t.start()

# 主线程等待 5 秒后设置事件
time.sleep(5)
event.set()
t.join()


Counter: 1
Counter: 2
Counter: 3
Counter: 4
Counter: 5
Worker stopped.


In [1]:
import threading
import time
import random

# 创建条件对象
condition = threading.Condition()
items = []
MAX_ITEMS = 5

def consumer(name):
    """消费者线程"""
    with condition:
        while len(items) == 0:
            print(f"🛒 消费者{name}: 没有商品，等待中...")
            condition.wait()  # 等待生产者通知
        
        item = items.pop(0)
        print(f"✅ 消费者{name}: 消费了 {item}")

def producer(name):
    """生产者线程"""
    for i in range(3):
        time.sleep(random.uniform(1, 2))  # 模拟生产时间
        
        with condition:
            if len(items) < MAX_ITEMS:
                item = f"商品-{name}-{i+1}"
                items.append(item)
                print(f"🏭 生产者{name}: 生产了 {item}")
                condition.notify()  # 通知等待的消费者
            else:
                print(f"📦 生产者{name}: 仓库已满，无法生产")

# 创建线程
consumers = [threading.Thread(target=consumer, args=(f"C{i+1}",)) for i in range(2)]
producers = [threading.Thread(target=producer, args=(f"P{i+1}",)) for i in range(2)]

# 启动线程
for t in consumers + producers:
    t.start()

# 等待所有线程完成
for t in consumers + producers:
    t.join()

print("🏁 所有线程完成")

🛒 消费者C1: 没有商品，等待中...
🛒 消费者C2: 没有商品，等待中...
🏭 生产者P1: 生产了 商品-P1-1
✅ 消费者C1: 消费了 商品-P1-1
🏭 生产者P2: 生产了 商品-P2-1
✅ 消费者C2: 消费了 商品-P2-1
🏭 生产者P1: 生产了 商品-P1-2
🏭 生产者P2: 生产了 商品-P2-2
🏭 生产者P1: 生产了 商品-P1-3
🏭 生产者P2: 生产了 商品-P2-3
🏁 所有线程完成


In [ ]:
import threading
import time

condition = threading.Condition()
items = []

def detailed_flow_demo():
    def consumer(name):
        print(f"👤 {name}: 尝试获取锁...")
        with condition:
            print(f"🔒 {name}: 成功获得锁")
            while len(items) == 0:
                print(f"😴 {name}: 没商品，释放锁并等待...")
                condition.wait()  # 释放锁，进入等待状态
                print(f"😊 {name}: 被唤醒，重新获得锁")
            
            item = items.pop(0)
            print(f"✅ {name}: 消费了 {item}")
        print(f"🔓 {name}: 释放锁，完成消费")
    
    def producer(name):
        time.sleep(1)  # 让消费者先开始
        print(f"🏭 {name}: 尝试获取锁...")
        with condition:
            print(f"🔒 {name}: 成功获得锁")
            item = f"商品-{name}"
            items.append(item)
            print(f"📦 {name}: 生产了 {item}")
            condition.notify()
            print(f"📢 {name}: 发送通知")
        print(f"🔓 {name}: 释放锁，完成生产")
    
    # 启动多个线程
    threads = []
    threads.append(threading.Thread(target=consumer, args=("消费者1",)))
    threads.append(threading.Thread(target=consumer, args=("消费者2",)))
    threads.append(threading.Thread(target=producer, args=("生产者1",)))
    
    for t in threads:
        t.start()
    for t in threads:
        t.join()

#detailed_flow_demo()

👤 消费者1: 尝试获取锁...
🔒 消费者1: 成功获得锁
😴 消费者1: 没商品，释放锁并等待...
👤 消费者2: 尝试获取锁...
🔒 消费者2: 成功获得锁
😴 消费者2: 没商品，释放锁并等待...
🏭 生产者1: 尝试获取锁...
🔒 生产者1: 成功获得锁
📦 生产者1: 生产了 商品-生产者1
📢 生产者1: 发送通知
🔓 生产者1: 释放锁，完成生产
😊 消费者1: 被唤醒，重新获得锁
✅ 消费者1: 消费了 商品-生产者1
🔓 消费者1: 释放锁，完成消费
